<a href="https://colab.research.google.com/github/joampc/TFM2023/blob/main/notebooks/ScrappingWeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ejecutar las dos siguientes celdas


In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF


Executing: /tmp/apt-key-gpghome.SQbyhvhgKa/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.tXSeQBtNjl/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.lRYQXbydXJ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip3 install selenium

from selenium import webdriver
from selenium.webdriver.chrome.options import Options



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://deb.debian.org/debian buster InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [684 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu foca

Aqui se conecta el driver de google chrome con la url de la pagina web de la base de datos

In [ ]:
url = "https://www.bedca.net/bdpub/index.php"
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.get(url)
print(driver.title)
#driver.quit()

Base de Datos BEDCA


**Codigo para obtener la tabla que tiene la lista de elementos**

COn este codigo obtenemos el listado de elementos que estan identificados por un ID.El fichero resultante se almacena en google drive por lo que te pedira acceso a la cuenta
tener en cuenta que en el campo ruta_archivo estamos definiendo una carpeta donde se almacenará el fichero csv resultante. Si no existe crearla o modificar ruta_archivo para la carpeta deseada

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from google.colab import drive
import csv

# Montar Google Drive
drive.mount('/content/drive')

# Ruta al directorio de Google Drive donde deseas guardar el archivo CSV
ruta_archivo = '/content/drive/MyDrive/Colab_Notebooks/resultados.csv'

# Crear un archivo CSV para escribir los datos
with open(ruta_archivo, 'w', newline='') as file:
    writer = csv.writer(file)

    # Escribir el encabezado en el archivo
    writer.writerow(['id|Nombre|English Name|Nombre Cientifico|Codigo foodex|Parte Comestible|Descripcion'])



    # Localiza el elemento por su xpath
    enlace_consulta = driver.find_element(By.XPATH, "//div[@class='link']/a[text()='Consulta']")

    # Haz clic en el enlace
    enlace_consulta.click()
    driver.implicitly_wait(10)

    # Esperar a que el enlace "Lista alfabética" esté visible y hacer clic en él
    enlace_alfabetico = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "Alfabetica")))
    enlace_alfabetico.click()

    # Esperar a que se carguen los enlaces de las letras del alfabeto
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "alphabet")))

    # Obtener la lista de enlaces de las letras del alfabeto
    enlaces_letras = driver.find_elements(By.CSS_SELECTOR, "#alphabet a")

    # Recorrer cada enlace y hacer clic
    for enlace_letra in enlaces_letras:
        # Obtener la letra correspondiente al enlace
        letra = enlace_letra.text.strip()

        # Hacer clic en el enlace de la letra
        enlace_letra.click()

        # Esperar a que se carguen los resultados (ajusta el tiempo de espera según sea necesario)
        driver.implicitly_wait(10)

        # Obtener referencia al div "content2"
        div_content2 = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "content2")))

        # Obtener referencia a la tabla de resultados dentro de "content2"
        table = div_content2.find_element(By.TAG_NAME, "table")

        # Obtener referencia a las filas de datos en la tabla
        table_rows = table.find_elements(By.XPATH, ".//tr[contains(@class, 'row-')]")

        # Recorrer las filas de datos y extraer el contenido
        for row_index in range(len(table_rows)):
            # Obtener las celdas de la fila actual
            cells = table_rows[row_index].find_elements(By.TAG_NAME, "td")
            row_data = "|".join([cell.text.strip() for cell in cells])

            # Obtener el enlace dentro de la celda
            enlace = cells[0].find_element(By.TAG_NAME, "a")
            # Hacer clic en el enlace para mostrar los detalles adicionales
            enlace.click()

            # Esperar a que se carguen los detalles
            WebDriverWait(driver, 10).until(EC.staleness_of(enlace))
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//table[tbody/tr/td/p[b[contains(., 'English name')]]]")))

            # Obtener referencia a la tabla de detalles
            detalles_table = driver.find_element(By.XPATH, "//table[tbody/tr/td/p[b[contains(., 'English name')]]]")

            # Obtener los elementos de texto dentro de la tabla de detalles
            detalles_elements = detalles_table.find_elements(By.TAG_NAME, "p")

            # Recorrer los elementos de texto y extraer su contenido
            detalles_values = [element.text.split(':', 1)[-1].strip() for element in detalles_elements[2:]]  # Excluir el campo "English name"
            detail_data = "|".join(detalles_values)
            print(f"{row_data} | {detail_data}")


            # Escribir los datos en el archivo CSV
            writer.writerow([row_data, detail_data])

            # Hacer clic en el enlace para volver al listado anterior
            volver_enlace = driver.find_element(By.XPATH, "//a[@id='Todos']")
            volver_enlace.click()
            # Esperar a que se cargue la tabla de listado anterior
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//table")))
            table = div_content2.find_element(By.TAG_NAME, "table")

            # Obtener referencia a las filas de datos en la tabla nuevamente
            table_rows = table.find_elements(By.XPATH, ".//tr[contains(@class, 'row-')]")

# Cerrar el navegador
driver.quit()


**Si fueramos a ejecutar el scrapping de otras paginas de la web, por ejemplo los componentes ejecutar nuevamente este codigo**


In [ ]:
url = "https://www.bedca.net/bdpub/index.php"
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.get(url)
print(driver.title)
#driver.quit()

Base de Datos BEDCA


**Este codigo hace scrapping de toda la lista de componentes, va iterando por cada una de las letras del alfabeto, es larga y se demora**
Si no fuera posible luego hay una solucion para scrappear y generar un archivo por cada letra

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import drive


# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

# Ruta al directorio de Google Drive donde deseas guardar el archivo CSV
ruta_archivo = '/content/drive/MyDrive/Colab_Notebooks/componentes.csv'




# Crear un archivo CSV para escribir los resultados
with open(ruta_archivo, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter='|')

    # Escribir la fila de encabezado
    writer.writerow(['ID', 'Grupo', 'Componente', 'Valor', 'Unidad', 'Fuente'])

    # Encontrar el enlace "Consulta" y hacer clic
    enlace_consulta = driver.find_element(By.XPATH, "//div[@class='link']/a[text()='Consulta']")
    enlace_consulta.click()
    driver.implicitly_wait(10)

    # Esperar a que el enlace "Lista alfabética" esté visible y hacer clic en él
    enlace_alfabetico = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "Alfabetica")))
    enlace_alfabetico.click()

    # Esperar a que se carguen los enlaces de las letras del alfabeto
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "alphabet")))

    # Obtener la lista de enlaces de las letras del alfabeto excluyendo el primero q dice Todos
    #enlaces_letras = driver.find_elements(By.CSS_SELECTOR, "#alphabet a")
    enlaces_letras = driver.find_elements(By.CSS_SELECTOR, "#alphabet a")[1:6]

    # Recorrer cada enlace y hacer clic
    for enlace_letra in enlaces_letras:
        # Obtener la letra correspondiente al enlace
        letra = enlace_letra.text.strip()

        # Hacer clic en el enlace de la letra
        enlace_letra.click()

        # Esperar a que se carguen los resultados
        driver.implicitly_wait(10)

        # Obtener referencia al div "content2"
        div_content2 = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "content2")))

        # Obtener referencia a la tabla de resultados dentro de "content2"
        table = div_content2.find_element(By.TAG_NAME, "table")

        # Obtener referencia a las filas de datos en la tabla
        table_rows = table.find_elements(By.XPATH, ".//tr[contains(@class, 'row-')]")

        # Recorrer las filas de datos y extraer el contenido
        for row_index in range(len(table_rows)):
            # Obtener las celdas de la fila actual
            cells = table_rows[row_index].find_elements(By.TAG_NAME, "td")
            id_elem = cells[0].text.strip()

            # Hacer clic en el enlace para mostrar los detalles adicionales
            enlace = cells[0].find_element(By.TAG_NAME, "a")
            enlace.click()
            # Esperar a que se cargue la tabla de composición
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//h4[text()='Información de composición (por 100 g de porción comestible)']")))

            # Encontrar el encabezado de la tabla de composición
            header = driver.find_element(By.TAG_NAME, "th")


            # Encontrar todos los elementos <tr> dentro de la tabla
            tbody = header.find_element(By.XPATH, "./ancestor::tbody")

            # Encontrar la primera fila de datos después del encabezado
            data_row = tbody.find_element(By.XPATH, "./tr[2]")


            # Obtener el contenido de la primera celda de la fila de datos
            first_cell = data_row.find_element(By.TAG_NAME, "td")
            grupo = first_cell.text
            # Obtener todos los elementos <tbody> después del encabezado
            tbody_elements = header.find_elements(By.XPATH, "../../following-sibling::tbody")



            # Extraer el contenido de cada fila

            for tbody_element in tbody_elements:
                # Obtener las filas de la tabla dentro del <tbody>
                tbody_rows = tbody_element.find_elements(By.TAG_NAME, "tr")

                for tbody_row in tbody_rows:
                    if 'background-color:#666600' in tbody_row.get_attribute('innerHTML'):
                        grupo = tbody_row.find_element(By.TAG_NAME, 'a').text
                        continue
                    composicion_alimento = []
                    cells = tbody_row.find_elements(By.TAG_NAME, "td")
                    componente = cells[0].text

                    valor = cells[1].text
                    unidad = cells[2].text
                    fuente = cells[3].text

                    # Agregar los datos a la lista de composición del alimento
                    composicion_alimento.append((id_elem, grupo, componente, valor, unidad, fuente))
                    print(f"{composicion_alimento}")
                    # Escribir la fila de datos en el archivo CSV
                    writer.writerow([id_elem, grupo, componente, valor, unidad, fuente])
        # Hacer clic en el enlace para volver al listado anterior

            volver_enlace = driver.find_element(By.XPATH, "//a[@id='Todos']")
            volver_enlace.click()



      # Volver a obtener la referencia a la tabla de resultados
            div_content2 = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "content2")))
            table = div_content2.find_element(By.TAG_NAME, "table")
            table_rows = table.find_elements(By.XPATH, ".//tr[contains(@class, 'row-')]")

        # Volver a la página anterior (opcional)
driver.back()


Este codigo genera un fichero por cada letra, la ventaja es que si falla se puede ejecutar nuevamente, y solamente se debe modificar en esta linea

**enlaces_letras = driver.find_elements(By.CSS_SELECTOR, "#alphabet a")[1:]**

el numero correspondiente a la posicion de la letra



In [ ]:
url = "https://www.bedca.net/bdpub/index.php"
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.get(url)
print(driver.title)
#driver.quit()

In [ ]:
from pickleshare import Path
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import drive




# Montar Google Drive
drive.mount('/content/drive')

# Ruta al directorio de Google Drive donde deseas guardar el archivo CSV
ruta_archivo = '/content/drive/MyDrive/Colab_Notebooks/'




# Encontrar el enlace "Consulta" y hacer clic
enlace_consulta = driver.find_element(By.XPATH, "//div[@class='link']/a[text()='Consulta']")
enlace_consulta.click()
driver.implicitly_wait(10)

# Esperar a que el enlace "Lista alfabética" esté visible y hacer clic en él
enlace_alfabetico = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "Alfabetica")))
enlace_alfabetico.click()

# Esperar a que se carguen los enlaces de las letras del alfabeto
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "alphabet")))

# Obtener la lista de enlaces de las letras del alfabeto excluyendo el primero que dice todos
#enlaces_letras = driver.find_elements(By.CSS_SELECTOR, "#alphabet a")
enlaces_letras = driver.find_elements(By.CSS_SELECTOR, "#alphabet a")[1:]


# Recorrer cada enlace y hacer clic
for enlace_letra in enlaces_letras:
    # Obtener la letra correspondiente al enlace
    nombre_archivo = ''
    letra = enlace_letra.text.strip()

    # Hacer clic en el enlace de la letra
    enlace_letra.click()
    print(letra)
    # Esperar a que se carguen los resultados
    driver.implicitly_wait(25)

    # Obtener referencia al div "content2"
    div_content2 = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "content2")))

    # Obtener referencia a la tabla de resultados dentro de "content2"
    table = div_content2.find_element(By.TAG_NAME, "table")

    # Obtener referencia a las filas de datos en la tabla
    table_rows = table.find_elements(By.XPATH, ".//tr[contains(@class, 'row-')]")


    # Crear un archivo CSV para escribir los resultados de la letra actual
    nombre_archivo = f'componentes_{letra}'
    path = f'{ruta_archivo}{nombre_archivo}.csv'
    with open(path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter='|')

        # Escribir la fila de encabezado
        writer.writerow(['ID', 'Grupo', 'Componente', 'Valor', 'Unidad', 'Fuente'])

        # Recorrer las filas de datos y extraer el contenido
        for row_index in range(len(table_rows)):
            # Obtener las celdas de la fila actual
            cells = table_rows[row_index].find_elements(By.TAG_NAME, "td")
            id_elem = cells[0].text.strip()

            # Hacer clic en el enlace para mostrar los detalles adicionales
            enlace = cells[0].find_element(By.TAG_NAME, "a")
            enlace.click()
            # Esperar a que se cargue la tabla de composición
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//h4[text()='Información de composición (por 100 g de porción comestible)']")))

            # Encontrar el encabezado de la tabla de composición
            header = driver.find_element(By.TAG_NAME, "th")

            # Encontrar todos los elementos <tr> dentro de la tabla
            tbody = header.find_element(By.XPATH, "./ancestor::tbody")

            # Encontrar la primera fila de datos después del encabezado
            data_row = tbody.find_element(By.XPATH, "./tr[2]")

            # Obtener el contenido de la primera celda de la fila de datos
            first_cell = data_row.find_element(By.TAG_NAME, "td")
            grupo = first_cell.text

            # Obtener todos los elementos <tbody> después del encabezado
            tbody_elements = header.find_elements(By.XPATH, "../../following-sibling::tbody")

            # Extraer el contenido de cada fila
            for tbody_element in tbody_elements:
                # Obtener las filas de la tabla dentro del <tbody>
                tbody_rows = tbody_element.find_elements(By.TAG_NAME, "tr")

                for tbody_row in tbody_rows:
                    if 'background-color:#666600' in tbody_row.get_attribute('innerHTML'):
                        grupo = tbody_row.find_element(By.TAG_NAME, 'a').text
                        continue
                    composicion_alimento = []
                    cells = tbody_row.find_elements(By.TAG_NAME, "td")
                    componente = cells[0].text

                    valor = cells[1].text
                    unidad = cells[2].text
                    fuente = cells[3].text

                    # Agregar los datos a la lista de composición del alimento
                    composicion_alimento.append((id_elem, grupo, componente, valor, unidad, fuente))
                    print(f"{composicion_alimento}")
                    # Escribir la fila de datos en el archivo CSV
                    writer.writerow([id_elem, grupo, componente, valor, unidad, fuente])

            # Hacer clic en el enlace para volver al listado anterior
            volver_enlace = driver.find_element(By.XPATH, "//a[@id='Todos']")
            volver_enlace.click()


            # Volver a obtener la referencia a la tabla de resultados
            div_content2 = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "content2")))
            table = div_content2.find_element(By.TAG_NAME, "table")
            table_rows = table.find_elements(By.XPATH, ".//tr[contains(@class, 'row-')]")

        # # Volver a la página anterior (opcional)
        # driver.back()

# Cerrar el controlador de Selenium
driver.quit()



**el siguiente codigo ejecutarlo por si se pierde la referencia a la pagina, igual si da error se puede volver a ejecutar las dos primeras celdas del notebook para volver a cargar el driver**






In [ ]:
url = "https://www.bedca.net/bdpub/index.php"
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.get(url)
print(driver.title)
#driver.quit()

Base de Datos BEDCA


**el codigo siguiente guarda en un fichero en drive la descricion de los componentes, tener en cuenta que debe existir la carpeta en drive como se recomendo previamente**

In [ ]:
from pickleshare import Path
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import drive
from selenium.common.exceptions import NoSuchElementException





# Montar Google Drive
drive.mount('/content/drive')

# Ruta al directorio de Google Drive donde deseas guardar el archivo CSV
ruta_archivo = '/content/drive/MyDrive/Colab_Notebooks/'




# Encontrar el enlace "Consulta" y hacer clic
enlace_consulta = driver.find_element(By.XPATH, "//div[@class='link']/a[text()='Consulta']")
enlace_consulta.click()
driver.implicitly_wait(10)

# Esperar a que el enlace "Lista alfabética" esté visible y hacer clic en él
enlace_alfabetico = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "Alfabetica")))
enlace_alfabetico.click()

# Esperar a que se carguen los enlaces de las letras del alfabeto
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "alphabet")))



enlaces_letras = driver.find_elements(By.CSS_SELECTOR, "#alphabet a")


# Obtener el enlace de la letra "M", busco la mayonesa con aceite de girasol porque tiene descripcion para todos los componentes
enlace_letra_m = enlaces_letras[13]  # Segundo enlace (índice 13) corresponde a la letra "M"

# Hacer clic en el enlace de la letra "M"
enlace_letra_m.click()

# Esperar a que se carguen los resultados
driver.implicitly_wait(30)

# Obtener referencia al div "content2"
div_content2 = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "content2")))

# Obtener referencia a la tabla de resultados dentro de "content2"
table = div_content2.find_element(By.TAG_NAME, "table")

# Obtener referencia a las filas de datos en la tabla
table_rows = table.find_elements(By.XPATH, ".//tr[contains(@class, 'row-')]")
first_row = table_rows[0]

# Obtener las celdas de la primera fila
cells = first_row.find_elements(By.TAG_NAME, "td")



for row in table_rows:
    try:
        cell = row.find_element(By.ID, "foodname")
        if cell.text == "Mayonesa con aceite de girasol":
            first_row = row

            break
    except NoSuchElementException:
        continue



# Crear un archivo CSV para escribir los resultados de la letra actual
nombre_archivo = f'descripcion_componentes'
path = f'{ruta_archivo}{nombre_archivo}.csv'
with open(path, 'w', newline='') as csvfile:
  writer = csv.writer(csvfile, delimiter='|')
  # Escribir la fila de encabezado
  writer.writerow(['componente', 'Descripcion'])
  # Obtener la primera fila de table_rows


  # Obtener las celdas de la primera fila
  cells = first_row.find_elements(By.TAG_NAME, "td")

  # Hacer clic en el enlace para mostrar los detalles adicionales
  enlace = cells[0].find_element(By.TAG_NAME, "a")
  enlace.click()

  # Esperar a que se cargue la tabla de composición
  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//h4[text()='Información de composición (por 100 g de porción comestible)']")))

  # Encontrar el encabezado de la tabla de composición
  header = driver.find_element(By.TAG_NAME, "th")

   # Obtener todos los elementos <tbody> después del encabezado
  tbody_elements = header.find_elements(By.XPATH, "../../following-sibling::tbody")

  # Extraer el contenido de cada fila
  for tbody_element in tbody_elements:
        # Obtener las filas de la tabla dentro del <tbody>
        tbody_rows = tbody_element.find_elements(By.TAG_NAME, "tr")
        for tbody_row in tbody_rows:
          if 'background-color:#666600' in tbody_row.get_attribute('innerHTML'):
              continue
          cells = tbody_row.find_elements(By.TAG_NAME, "td")
          componente = cells[0].text
          script = f"var element = document.getElementById('glos{componente}'); if (element) return element.textContent;"
          contenido_div = driver.execute_script(script)
          if contenido_div is None:
            contenido_div = ''

          # Imprimir el contenido del elemento <div>
          print(f"{componente}|{contenido_div}")

          # Escribir la fila de datos en el archivo CSV
          writer.writerow([componente, contenido_div])











# Cerrar el controlador de Selenium
#driver.quit()


In [ ]:
url = "https://www.bedca.net/bdpub/index.php"
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.get(url)
print(driver.title)
#driver.quit()

Base de Datos BEDCA


**Aqui se genera un archivo con el contenido de las fuentes de donde se ha obtenido la informacion de composicion**

In [ ]:
from pickleshare import Path
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import drive
from selenium.common.exceptions import NoSuchElementException





# Montar Google Drive
drive.mount('/content/drive')

# Ruta al directorio de Google Drive donde deseas guardar el archivo CSV
ruta_archivo = '/content/drive/MyDrive/Colab_Notebooks/'




# Encontrar el enlace "Consulta" y hacer clic
enlace_fuentes = driver.find_element(By.XPATH, "//div[@class='link']/a[text()='Fuentes']")
enlace_fuentes.click()


# Esperar a que se carguen los resultados
driver.implicitly_wait(30)

# Obtener referencia al div "content2"
div_content2 = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "content2")))

# Obtener referencia a la tabla de resultados dentro de "content2"
table = div_content2.find_element(By.TAG_NAME, "table")

# Obtener referencia a las filas de datos en la tabla
table_rows = table.find_elements(By.XPATH, ".//tr[contains(@class, 'row-')]")

nombre_archivo = f'Fuentes'
path = f'{ruta_archivo}{nombre_archivo}.csv'
with open(path, 'w', newline='') as csvfile:
  writer = csv.writer(csvfile, delimiter='|')

  # Escribir la fila de encabezado
  writer.writerow(['Id_fuente', 'Fuente'])
  for row in table_rows:
      # Obtener las celdas de la fila actual
      cells = row.find_elements(By.TAG_NAME, "td")
      id_elem = cells[0].text.strip()
      fuente = cells[1].text.strip()
      # Imprimir el contenido del elemento <div>
      print(f"{id_elem}|{fuente}")

      # Escribir la fila de datos en el archivo CSV
      writer.writerow([id_elem, fuente])




In [ ]:
url = "https://www.bedca.net/bdpub/index.php"
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.get(url)
print(driver.title)
#driver.quit()

Base de Datos BEDCA


In [ ]:
from pickleshare import Path
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import drive
from selenium.webdriver.support.ui import Select





# Montar Google Drive
drive.mount('/content/drive')

# Ruta al directorio de Google Drive donde deseas guardar el archivo CSV
ruta_archivo = '/content/drive/MyDrive/Colab_Notebooks/'




# Encontrar el enlace "Consulta" y hacer clic
enlace_consulta = driver.find_element(By.XPATH, "//div[@class='link']/a[text()='Consulta']")
enlace_consulta.click()
driver.implicitly_wait(10)

# Esperar a que el enlace "Grupo de Alimentos" esté visible y hacer clic en él
enlace_grupo = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "General")))
enlace_grupo.click()

driver.implicitly_wait(30)


# # Obtener referencia al elemento select
# select_element = driver.find_element(By.ID, "fglist")

# # Crear objeto Select a partir del elemento select
# select = Select(select_element)

# Esperar a que el elemento del select esté visible
wait = WebDriverWait(driver, 10)
select_element = wait.until(EC.visibility_of_element_located((By.ID, "fglist")))

# Crear el objeto Select
select = Select(select_element)

# Esperar a que el elemento del select esté visible
wait = WebDriverWait(driver, 10)
select_element = wait.until(EC.visibility_of_element_located((By.ID, "fglist")))

filename = "grupos_alimentos"
path = f'{ruta_archivo}{filename}.csv'

with open(path, 'w', newline='') as csvfile:
   writer = csv.writer(csvfile)
   writer.writerow(["ID Select", "Texto", "ID"])

    # Recorrer cada opción y seleccionarla
   for i in range(len(select_element.find_elements(By.TAG_NAME, "option"))):
        # Obtener el elemento del select en cada iteración para evitar StaleElementReferenceException
        select_element = wait.until(EC.visibility_of_element_located((By.ID, "fglist")))

        # Crear el objeto Select
        select = Select(select_element)

        # Obtener la opción actual en cada iteración
        option = select.options[i]

        # Obtener el texto de la opción
        texto = option.text

        # Seleccionar la opción en el select
        select.select_by_index(i)
        # texto=option.text
        print(texto)

        # Obtener el valor de la opción
        value = option.get_attribute("value")

        # # Seleccionar la opción en el select
        # select.select_by_value(value)


        # Seleccionar la opción en el select
        select.select_by_index(i)

        # Obtener el valor de la opción seleccionada
        selected_value = select.first_selected_option.get_attribute("value")
        # Hacer clic en el botón "Consultar"
        consultar_button = driver.find_element(By.XPATH, "//input[@value='Consultar']")
        consultar_button.click()


        # Esperar a que se cargue la tabla "querytable1"
        table = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, "querytable1")))

        # Obtener referencia a las filas de la tabla
        #rows = table.find_elements(By.TAG_NAME, "tr")
        rows = table.find_elements(By.XPATH, ".//tr[contains(@class, 'row-')]")


        # Crear un archivo CSV para escribir los datos de la tabla
        filename = f"grupos.csv"
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["ID Select", "Texto", "ID"])

            # Recorrer las filas de la tabla y guardar los datos en el archivo CSV
            for row in rows:
                # Obtener las celdas de la fila actual
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 1:
                    id_select = value
                    #id_select = select_element.get_attribute("value")

                    id_celda = cells[0].text.strip()
                    print(f"{id_select}|{texto}|{id_celda}")
                    writer.writerow([id_select, texto, id_celda])
            # Hacer clic en el enlace para volver al listado anterior
            volver_enlace = driver.find_element(By.XPATH, "//a[@id='Todos']")
            volver_enlace.click()

In [76]:
!ls



notebooks  README.md


In [77]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [78]:
!git add notebooks


In [79]:
!git commit -m "Agregada la carpeta notebooks"


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [81]:
!git push origin main


Load key "/content/id_rsa.pub": invalid format
git@github.com: Permission denied (publickey).
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
